In [1]:
from ipywidgets import VBox, Button, FileUpload, Output, Label
from PIL import Image
from IPython.display import display
import io
import numpy as np
import torchvision
import torchvision.transforms as T
import torch


# Decide which model you want to use among the ones exported
learn_inf = torch.jit.load("checkpoints/transfer_exported.pt" )

def on_click_classify(change):
    
  

    
   
    out_pl.clear_output()  # Clear previous output
    
    # Check if any file was uploaded
    if not btn_upload.value:
        with out_pl:
            display(Label("No file uploaded. Please select an image."))
        return
    
    
    uploaded_file_info = btn_upload.value[0] # Access the first dictionary in the tuple
    fn = io.BytesIO(uploaded_file_info['content'])
    
    img = Image.open(fn).convert('RGB') # Ensure image is in RGB format

    # Display the image
    with out_pl:

        ratio = img.size[0] / img.size[1]
        c = img.copy()
        width = 300
        height= int(width/ratio)
        c.thumbnail([width, height])
        display(c)
        
 
    

    timg = T.ToTensor()(img).unsqueeze_(0)
    
        # Calling the model
    softmax = learn_inf(timg).data.cpu().numpy().squeeze()
    
    # Get the indexes of the classes ordered by softmax
    # (larger first)
    idxs = np.argsort(softmax)[::-1]
    
    # Loop over the classes with the largest softmax
    for i in range(5):
        # Get softmax value
        p = softmax[idxs[i]]
    
        # Get class name
        landmark_name = learn_inf.class_names[idxs[i]]
        
        labels[i].value = f"{landmark_name} (prob: {p:.2f})"


# Putting back btn_upload to a widget for next cell
btn_upload = FileUpload()

btn_run = Button(description="Classify")
btn_run.on_click(on_click_classify)

labels = []
for _ in range(5):
    labels.append(Label())

out_pl = Output()
out_pl.clear_output()

wgs = [Label("Please upload a picture of a landmark"), btn_upload, btn_run, out_pl]
wgs.extend(labels)

VBox(wgs)







    
    
  
    

